#Chapter 5. 문장 쌍 분류하기  

문장 쌍 분류란 문장(또는 문서)가 2개가 주어졌을 때 해당 문장 사이의 관계가 어떤 범주일지 분류하는 과제입니다. 문장 쌍 분류의 대표 예시로 자연어 추론이 있습니다. 자연어 추론은 두 문장의 관계가 참인지, 거짓인지, 중립 또는 판단 불가인지 가려내는 것입니다.  
이 장에서는 인공지능 기업 '업스테이지'가 공개한 KLUE-NLI 데이터셋을 가지고 문장 쌍 분류 모델을 구축하는 방법을 살펴봅니다.

##<5-1 문장 쌍 분류 모델 훑어보기>  

이 장에서는 ***문장 쌍 분류(sentence pair classification)*** 의 대표 예로 ***자연어 추론(natural language inference, NLI)*** 과제를 실습합니다. 본격적인 실습에 앞서 모델의 아키텍처와 입출력 등 전반을 살펴보겠습니다.

자연어 추론은 2개의 문장(또는 문서)이 참(entailment), 거짓(contradiction), 중립 또는 판단 불가(neutral)인지 가려내는 것입니다. 여기에서 **entailment**는 '함의', **contradiction**은 '모순'으로 번역되기도 합니다. 예를 들면 다음과 같습니다.  

> 나 출근했어 + 난 백수야 $\rightarrow$ **거짓**  
> 나 출근했어 + 난 개발자다 $\rightarrow$ **중립**  

이번 실습에서 사용할 데이터는 업스테이지에서 공개한 NLI 데이터셋입니다. 이 데이터셋은 전제(premise)에 대한 가설(hypothesis)이 참인지, 거짓인지, 중립인지 정보가 레이블(**gole_label**)로 주어져 있습니다.

<center><표 5-1 NLI 데이터셋 예시></center>

|-|-|-|-|
|------|------|---|---|
|**전제**|100분간 잘껄 그래도 소닉붐땜에 2점준다|100분간 잘껄 그래도 소닉붐땜에 2점준다|101빌딩 근처에 나름 즐길거리가 많습니다.|
|**가설**|100분간 잤다.|소닉붐이 정말 멋있었다.|101빌딩 부근에서는 여러가지를 즐길수 있습니다.|
|**레이블**|contradiction|neutral|entailment|


우리가 만들 NLI 과제 수행 모델은 전제와 가설 2개 문장을 입력으로 하고, 두 문장의 관계가 어떤 범주일지 확률을 출력으로 합니다. 그리고 출력을 적당한 후처리 과정을 거쳐 참(entailment), 거짓(contradiction), 중립(neutral)등 사람이 보기에 좋은 형태로 가공해 줍니다.  
예를 들어 다음과 같습니다.

- 100분간 잘껄 그래도 소닉붐땜에 2점준다**(전제)** + 100분간 잤다.**(가설)** → [0.02, 0.97, 0.01]**(참, 거짓, 중립일 확률)** → contradiction**(후처리 결과)**
- 100분간 잘껄 그래도 소닉붐땜에 2점준다**(전제)** + 소닉붐이 정말 멋있었다.**(가설)** → [0.01, 0.01, 0.98]**(참, 거짓, 중립일 확률)** → neutral**(후처리 결과)**

### 모델 구조  
---
이 책에서 사용하는 문장 쌍 분류 모델은 전제와 가설 두 문장을 각각 토큰화한 뒤 **[CLS] + 전제 + [SEP] + 가설 + [SEP]** 형태로 이어 붙입니다. 여기에서 **CLS**는 문장 시작을 알리는 스페셜 토큰, **SEP**는 전제와 가설을 서로 구분해 주는 스페셜 토큰입니다.

<center><그림 5-1 문장 쌍 분류></center>

<p align="center"><img src="https://i.imgur.com/Wd9UK1s.png">  

<center>출처 : ratsgo's NLPBOOK</center>

이를 BERT 모델에 입력하고 문장 수준의 벡터(**pooler_output**)를 뽑습니다. 이 벡터엔 전제와 가설의 의미가 응축되어 있습니다. 여기에 작은 추가 모듈을 덧붙여 모델 전체의 출력이 [전제에 대해 가설이 참일 확률, 전제에 대해 가설이 거짓일 확률, 전제에 대해 가설이 중립일 확률] 형태가 되도록 합니다.

### 태스크 모듈
---
**pooler_output** 벡터 뒤에 붙는 추가 모듈의 구조는 다음 그림과 같습니다. 우선 **pooler_output** 벡터(그림에서 $x$)에 드롭아웃을 적용합니다. 그다음 가중치 행렬을 곱해 **pooler_output**을 분류해야 할 범주 수만큼의 차원을 갖는 벡터로 변환합니다(그림에서 $h$). 만일 **pooler_output** 벡터가 768차원이고 분류 대상 범주 수가 3개(참, 거짓, 중립)라면 가중치 행렬 크기는 768 $\times$ 3이 됩니다.

<center><그림 5-2 문장 쌍 분류 태스크 모듈></center>

<p align="center"><img src="https://i.imgur.com/qub0zK2.png">  

<center>출처 : ratsgo's NLPBOOK</center>


여기에 소프트맥스 함수를 취하면 모델의 최종 출력(그림에서 $y$)이 됩니다. 이렇게 만든 모델의 최종 출력과 정답 레이블을 비교해 모델 출력이 정답 레이블과 최대한 같아지도록 BERT 레이어를 포함한 모델 전체를 업데이트합니다.

문장 쌍 분류 태스크 모듈은 4장에서 다른 문서 분류 태스크 모듈과 거의 유사한 모습입니다.  
문서 분류 과제를 3개 범주(긍정, 부정, 중립)를 분류하는 태스크로 상정한다면 두 모듈 구조는 똑같습니다. 다만 차이는 태스크 모듈의 입력(**pooler_output**)이 됩니다. 즉, **pooler_output**에 문장 1개의 의미가 응축되어 있다면 문서 분류, 2개의 의미가 내포해 있다면 문장 쌍 분류 과제가 됩니다.